In [1]:
"""
主要是参考internthinker的能力，经过非微调的方式解决现有问题
参考：https://mp.weixin.qq.com/s/AbL3KewLXtvweEk9seOTAw
实现方式：借助于size较大的LLM，通过Prompt进行实现。

"""

import os

In [2]:


from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)



In [25]:

def restate(query):

    system_prompt = """你是一个AI专家助手，擅长对问题进行解析，规划问题的解决思路，能够以严密的逻辑以层层递进的关系经过推理重述问题的解决步骤，并最终完成对问题的解析。
    - 重述问题的解决思路放入到<restate></restate>标志中
    - 根据重述的问题，总结出解决步骤的关键步骤，形成解决问题的子问题列表，每个子问题以#号标志开头，子问题尽量以动词等关键词开头，,所有的子问题列表放入到<subquery></subquery>标志中。
    - 不要输出其它无关的内容，仅输出<restate>和<subquery>标记内的内容
    - 不要随意猜想，不要输出与原始问题无关的内容
    - 子问题以列表的形式返回
    - 子问题是从原始问题中语义解析和语义抽取出来的

    """

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": Query.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content
    
    

    

query = "姚明和奥尼尔谁的身高高"
query = "张三的父亲叫什么"
# query = "功率最大的发电机是什么时候购买的"
# query = "耗电量最大的设备是什么？为什么它耗电量这么大"
# query = "耗电量最大的设备名称是及原因"
# query= "张三购物费用比旅游费用多了多少"


print(restate(query))



<restate>问题要求找出张三的父亲的名字。</restate>
<subquery>#获取关于张三的信息  
#从信息中找到张三父亲的名字</subquery>


In [76]:
# r  = """
# <subquery>
# - 确定张三的购物费用
# - 确定张三的旅游费用
# - 计算购物费用与旅游费用的差值</subquery>
# <restate>确定张三购物费用和旅游费用的具体数值，然后计算两者之间的差值。</restate>"""

# import re


# restate_pattern = "<restate>(?P<restate>.*?)</restate>.*?<subquery>(?P<subquery>.*?</subquery>)"
# subquery_pattern = "<subquery>(?P<subquery>.*?)</subquery>"

# resate_matcher = re.search(restate_pattern, r, re.S)
# subquery_matcher = re.search(subquery_pattern, r, re.S)

# print(resate_matcher.group("restate"))
# print(subquery_matcher.group("subquery"))



AttributeError: 'NoneType' object has no attribute 'group'

In [74]:

r  = """<restate>确定张三购物费用和旅游费用的具体数值，然后计算两者之间的差值。</restate>
<subquery>
- 确定张三的购物费用
- 确定张三的旅游费用
- 计算购物费用与旅游费用的差值</subquery>
"""

pattern = "<restate>(?P<restate>[\\s\\S]*?)</restate>"

print(re.search(pattern, r).group("restate"))


确定张三购物费用和旅游费用的具体数值，然后计算两者之间的差值。
